In [2]:
import requests
import base64
import json
import os
from docx import Document

# WordPress 사이트 설정
WP_URL = "https://prayer-church.co.kr"  # WordPress 사이트 URL
WP_API = f"{WP_URL}/wp-json/wp/v2"
USERNAME = "prayerchurch"  # WordPress 사용자명
APP_PASSWORD = "AziD L62i Ga4b 1efW ToAs hBMB"  # 응용프로그램 비밀번호

# 문서 경로
DOC_PATH = "/Users/popi/Desktop/wp/files/word/추수감사 주일 대표 기도문.docx"
POST_TITLE = "추수감사 주일 대표 기도문"
CATEGORY_NAME = "주일 대표 기도문"

def read_docx_file(file_path):
    """docx 파일을 읽어 내용을 반환합니다."""
    try:
        doc = Document(file_path)
        full_text = []
        
        for para in doc.paragraphs:
            full_text.append(para.text)
            
        # 문단 사이에 줄바꿈 추가
        return "\n\n".join(full_text)
    except Exception as e:
        print(f"문서 읽기 오류: {e}")
        return None

def get_category_id(category_name):
    """카테고리 이름으로 ID를 찾습니다."""
    try:
        # 모든 카테고리 가져오기
        response = requests.get(f"{WP_API}/categories")
        categories = response.json()
        
        # 일치하는 카테고리 찾기
        for category in categories:
            if category["name"].lower() == category_name.lower():
                return category["id"]
        
        print(f"카테고리 '{category_name}'을(를) 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"카테고리 ID 검색 오류: {e}")
        return None

def create_post(title, content, category_id):
    """WordPress에 새 글을 작성합니다."""
    # 인증 설정
    credentials = f"{USERNAME}:{APP_PASSWORD}"
    token = base64.b64encode(credentials.encode()).decode()
    headers = {
        "Authorization": f"Basic {token}",
        "Content-Type": "application/json"
    }
    
    # 포스트 데이터 준비
    post_data = {
        "title": title,
        "content": content,
        "status": "publish",  # 바로 발행
    }
    
    # 카테고리 ID가 있으면 추가
    if category_id:
        post_data["categories"] = [category_id]
    
    try:
        # 포스트 생성 요청
        response = requests.post(
            f"{WP_API}/posts", 
            headers=headers, 
            data=json.dumps(post_data)
        )
        
        if response.status_code == 201:  # 201: Created
            post_info = response.json()
            print(f"글이 성공적으로 게시되었습니다: {post_info['link']}")
            return True
        else:
            print(f"글 게시 실패: {response.status_code}")
            print(f"오류 메시지: {response.text}")
            return False
    except Exception as e:
        print(f"글 게시 중 오류 발생: {e}")
        return False

def main():
    # docx 파일 읽기
    content = read_docx_file(DOC_PATH)
    if not content:
        print("문서를 읽을 수 없습니다. 경로를 확인해주세요.")
        return
    
    # 카테고리 ID 가져오기
    category_id = get_category_id(CATEGORY_NAME)
    
    # 글 게시하기
    create_post(POST_TITLE, content, category_id)

if __name__ == "__main__":
    main()

글이 성공적으로 게시되었습니다: https://prayer-church.co.kr/%ec%b6%94%ec%88%98%ea%b0%90%ec%82%ac-%ec%a3%bc%ec%9d%bc-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8/


In [3]:
import requests
import base64
import json
import os
from docx import Document
from docx.enum.style import WD_STYLE_TYPE

# WordPress 사이트 설정
WP_URL = "https://prayer-church.co.kr"  # WordPress 사이트 URL
WP_API = f"{WP_URL}/wp-json/wp/v2"
USERNAME = "prayerchurch"  # WordPress 사용자명
APP_PASSWORD = "AziD L62i Ga4b 1efW ToAs hBMB"  # 응용프로그램 비밀번호

# 문서 경로
DOC_PATH = "/Users/popi/Desktop/wp/files/word/총동원 주일 기도문.docx"
POST_TITLE = "총동원 주일 기도문"
CATEGORY_NAME = "주일 대표 기도문"

def get_heading_level(para):
    """문단의 제목 레벨을 확인합니다. (Heading 1, Heading 2 등)"""
    if not para.style:
        return None
    
    # 스타일 이름을 확인
    style_name = para.style.name.lower()
    
    # 제목 스타일 확인
    if "heading 1" in style_name or "제목 1" in style_name:
        return 1
    elif "heading 2" in style_name or "제목 2" in style_name:
        return 2
    elif "heading 3" in style_name or "제목 3" in style_name:
        return 3
    elif "heading 4" in style_name or "제목 4" in style_name:
        return 4
    elif "heading 5" in style_name or "제목 5" in style_name:
        return 5
    elif "heading 6" in style_name or "제목 6" in style_name:
        return 6
    
    return None

def read_docx_file_with_formatting(file_path):
    """docx 파일을 읽어 HTML 형식으로 내용을 반환합니다."""
    try:
        doc = Document(file_path)
        html_content = []
        
        for para in doc.paragraphs:
            text = para.text.strip()
            if not text:
                html_content.append("<p>&nbsp;</p>")  # 빈 줄 추가
                continue
                
            # 제목 확인
            heading_level = get_heading_level(para)
            
            # 서식 적용
            if heading_level:
                html_content.append(f"<h{heading_level}>{text}</h{heading_level}>")
            else:
                # 기본 단락
                html_content.append(f"<p>{text}</p>")
            
        return "\n".join(html_content)
    except Exception as e:
        print(f"문서 읽기 오류: {e}")
        return None

def get_category_id(category_name):
    """카테고리 이름으로 ID를 찾습니다."""
    try:
        # 모든 카테고리 가져오기
        response = requests.get(f"{WP_API}/categories")
        categories = response.json()
        
        # 일치하는 카테고리 찾기
        for category in categories:
            if category["name"].lower() == category_name.lower():
                return category["id"]
        
        print(f"카테고리 '{category_name}'을(를) 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"카테고리 ID 검색 오류: {e}")
        return None

def create_post(title, content, category_id):
    """WordPress에 새 글을 작성합니다."""
    # 인증 설정
    credentials = f"{USERNAME}:{APP_PASSWORD}"
    token = base64.b64encode(credentials.encode()).decode()
    headers = {
        "Authorization": f"Basic {token}",
        "Content-Type": "application/json"
    }
    
    # 포스트 데이터 준비
    post_data = {
        "title": title,
        "content": content,
        "status": "publish",  # 바로 발행
    }
    
    # 카테고리 ID가 있으면 추가
    if category_id:
        post_data["categories"] = [category_id]
    
    try:
        # 포스트 생성 요청
        response = requests.post(
            f"{WP_API}/posts", 
            headers=headers, 
            data=json.dumps(post_data)
        )
        
        if response.status_code == 201:  # 201: Created
            post_info = response.json()
            print(f"글이 성공적으로 게시되었습니다: {post_info['link']}")
            return True
        else:
            print(f"글 게시 실패: {response.status_code}")
            print(f"오류 메시지: {response.text}")
            return False
    except Exception as e:
        print(f"글 게시 중 오류 발생: {e}")
        return False

def main():
    # docx 파일 읽기 (HTML 형식으로)
    content = read_docx_file_with_formatting(DOC_PATH)
    if not content:
        print("문서를 읽을 수 없습니다. 경로를 확인해주세요.")
        return
    
    # 카테고리 ID 가져오기
    category_id = get_category_id(CATEGORY_NAME)
    
    # 글 게시하기
    create_post(POST_TITLE, content, category_id)

if __name__ == "__main__":
    main()

글이 성공적으로 게시되었습니다: https://prayer-church.co.kr/%ec%b4%9d%eb%8f%99%ec%9b%90-%ec%a3%bc%ec%9d%bc-%ea%b8%b0%eb%8f%84%eb%ac%b8/


In [4]:
import requests
import base64
import json
import os
import re
import pandas as pd

# WordPress 사이트 설정
WP_URL = "https://prayer-church.co.kr"  # WordPress 사이트 URL
WP_API = f"{WP_URL}/wp-json/wp/v2"
USERNAME = "prayerchurch"  # WordPress 사용자명
APP_PASSWORD = "AziD L62i Ga4b 1efW ToAs hBMB"  # 응용프로그램 비밀번호

# 엑셀 파일 경로
EXCEL_PATH = "/Users/popi/Desktop/wp/post/post_list.xlsx"

def read_excel_posts():
    """엑셀 파일에서 게시물 데이터를 읽어옵니다."""
    try:
        # 엑셀 파일 읽기
        df = pd.read_excel(EXCEL_PATH)
        
        # 필요한 컬럼이 있는지 확인
        required_columns = ['title', 'category', 'body']
        for col in required_columns:
            if col not in df.columns:
                print(f"오류: 엑셀 파일에 '{col}' 열이 없습니다.")
                return []
        
        # 데이터 추출
        posts = []
        for _, row in df.iterrows():
            post = {
                'title': str(row['title']),
                'category': str(row['category']),
                'body': str(row['body'])
            }
            posts.append(post)
        
        print(f"{len(posts)}개의 게시물 데이터를 읽었습니다.")
        return posts
    except Exception as e:
        print(f"엑셀 파일 읽기 오류: {e}")
        return []

def format_content(body):
    """본문 내용을 형식화합니다. <h2> 태그를 실제 HTML h2 태그로 변환"""
    # <h2> 태그 처리
    content = re.sub(r'<h2>(.*?)</h2>', r'<h2>\1</h2>', body)
    
    # 일반 텍스트는 p 태그로 감싸기
    # 먼저 이미 태그로 감싸인 텍스트를 제외
    paragraphs = []
    text_parts = re.split(r'(<h[1-6]>.*?</h[1-6]>)', content)
    
    for part in text_parts:
        if part.strip():
            # 이미 h 태그로 감싸인 부분은 그대로 유지
            if part.startswith('<h') and part.endswith('</h'):
                paragraphs.append(part)
            else:
                # 일반 텍스트 줄바꿈 처리
                lines = part.split('\n')
                for line in lines:
                    if line.strip():
                        paragraphs.append(f"<p>{line.strip()}</p>")
    
    return '\n'.join(paragraphs)

def get_category_id(category_name):
    """카테고리 이름으로 ID를 찾습니다."""
    try:
        # 모든 카테고리 가져오기
        response = requests.get(f"{WP_API}/categories")
        categories = response.json()
        
        # 일치하는 카테고리 찾기
        for category in categories:
            if category["name"].lower() == category_name.lower():
                return category["id"]
        
        print(f"카테고리 '{category_name}'을(를) 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"카테고리 ID 검색 오류: {e}")
        return None

def create_post(title, content, category_name):
    """WordPress에 새 글을 작성합니다."""
    # 카테고리 ID 가져오기
    category_id = get_category_id(category_name)
    
    # 인증 설정
    credentials = f"{USERNAME}:{APP_PASSWORD}"
    token = base64.b64encode(credentials.encode()).decode()
    headers = {
        "Authorization": f"Basic {token}",
        "Content-Type": "application/json"
    }
    
    # 포스트 데이터 준비
    post_data = {
        "title": title,
        "content": content,
        "status": "publish",  # 바로 발행
    }
    
    # 카테고리 ID가 있으면 추가
    if category_id:
        post_data["categories"] = [category_id]
    
    try:
        # 포스트 생성 요청
        response = requests.post(
            f"{WP_API}/posts", 
            headers=headers, 
            data=json.dumps(post_data)
        )
        
        if response.status_code == 201:  # 201: Created
            post_info = response.json()
            print(f"글 '{title}' 성공적으로 게시되었습니다: {post_info['link']}")
            return True
        else:
            print(f"글 '{title}' 게시 실패: {response.status_code}")
            print(f"오류 메시지: {response.text}")
            return False
    except Exception as e:
        print(f"글 '{title}' 게시 중 오류 발생: {e}")
        return False

def main():
    # 엑셀 파일에서 게시물 데이터 읽기
    posts = read_excel_posts()
    
    if not posts:
        print("게시할 데이터가 없습니다.")
        return
    
    # 각 게시물 처리
    for post in posts:
        title = post['title']
        category = post['category']
        body = post['body']
        
        # 본문 형식화
        formatted_content = format_content(body)
        
        # 글 게시하기
        print(f"'{title}' 게시물을 처리 중...")
        create_post(title, formatted_content, category)

if __name__ == "__main__":
    main()

1개의 게시물 데이터를 읽었습니다.
'청년 주일 대표 기도문' 게시물을 처리 중...
글 '청년 주일 대표 기도문' 성공적으로 게시되었습니다: https://prayer-church.co.kr/%ec%b2%ad%eb%85%84-%ec%a3%bc%ec%9d%bc-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8/
